In [1]:
import numpy as np
import pandas as pd
# from source.Files_operating import read_sht_data

shtRipper v1.3


In [45]:
F_ID = 44375
# 0 - start inds, 1 - end inds, 2 - marks
marks = [[], [], []]

In [46]:
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator

if l_edge == "":
    l_edge = 0
else:
    l_edge = int(l_edge)


# print("---")
step = 3000
width = 5000

step_away = 10

res = list(map(int, input(f"Input fragments info: start ind | end ind | mark (1 - dELM, 2 - LCO, 3 - EHO)\n").strip().split()))

while len(res) >= 3:
    
    marks[0].append(res[0])
    marks[1].append(res[1])
    marks[2].append(res[2])

    res = list(map(int, input(f"Input fragments info:\n").strip().split()))

# Marks: 1 - dELM, 2 - LCO, 3 - EHO
    
# 217500

Input fragments info: start ind | end ind | mark (1 - dELM, 2 - LCO, 3 - EHO)
 144000 146370 3
Input fragments info:
 146400 148000 3
Input fragments info:
 148210 149150 3
Input fragments info:
 151040 153945 3
Input fragments info:
 154265 159664 2
Input fragments info:
 160230 161020 2
Input fragments info:
 162380 163225 2
Input fragments info:
 163820 165450 2
Input fragments info:
 171440 172740 2
Input fragments info:
 173400 174700 1
Input fragments info:
 175685 176350 1
Input fragments info:
 177430 178395 1
Input fragments info:
 178980 180685 1
Input fragments info:
 181675 182160 1
Input fragments info:
 183500 184850 1
Input fragments info:
 


In [47]:
marks_df = pd.DataFrame(data={"s_i": marks[0], "e_i": marks[1], "m": marks[2]})
marks_df.head(10)

,s_i,e_i,m
0,144000,146370,3
1,146400,148000,3
2,148210,149150,3
3,151040,153945,3
4,154265,159664,2
5,160230,161020,2
6,162380,163225,2
7,163820,165450,2
8,171440,172740,2
9,173400,174700,1


In [48]:
# D:/Edu/Lab/Projects/Plasma_analysis/data/sht/G-ELM/ | C:/Users/f.belous/Work/Projects/Plasma_analysis/data/sht/G-ELM/
file_path = "C:/Users/f.belous/Work/Projects/Plasma_analysis/data/df/marks/"
marks_df.to_csv(file_path + f"{F_ID}_marks.csv", index=False)

---
---

In [35]:
F_ID = 44187
dir_path = "C:/Users/f.belous/Work/Projects/Plasma_analysis/data/sht/NG-ELM/"  
# D:/Edu/Lab/Projects/Plasma_analysis/data/sht/G-ELM/ | C:/Users/f.belous/Work/Projects/Plasma_analysis/data/sht/G-ELM/

# df = read_sht_data(f'sht{F_ID}', dir_path)
# df = df.rename(columns={"ch1": "d_alpha"})

# df["mark_dELM"] = 0
# df["mark_LCO"] = 0
# df["mark_EHO"] = 0

# df.describe()

,t,d_alpha,mark_dELM,mark_LCO,mark_EHO
count,393216.000000,393216.000000,393216.0,393216.0,393216.0
mean,0.196608,0.288921,0.0,0.0,0.0
std,0.113512,0.421453,0.0,0.0,0.0
min,0.000000,-0.408984,0.0,0.0,0.0
25%,0.098304,0.015337,0.0,0.0,0.0
50%,0.196607,0.066460,0.0,0.0,0.0
75%,0.294911,0.570022,0.0,0.0,0.0
max,0.393215,4.887363,0.0,0.0,0.0


In [5]:
mgd_v = read_sht_data(f'sht{F_ID}', dir_path, data_name="МГД быстрый зонд верт.").ch1.to_numpy()
# mgd_data_radial
mgd_r = read_sht_data(f'sht{F_ID}', dir_path, data_name="МГД быстрый зонд рад.").ch1.to_numpy()

In [ ]:


# mode_plotting = 0  # int(input("Input visualising mode [ 0 - manual | 1 - only marked ]:"))
# mode_marking = int(input("Input marking mode (0 - manual | 1 - semiauto): ").strip().split()[0])
# l_edge = input(f"Input start index [0:{df.shape[0]//1000*1000}]: ").strip().split()[0]
    
    # if mode_plotting == 1 and sum(mark_d_alpha[l_edge:r_edge]) == 0 and sum(mark_sxr[l_edge:r_edge]) == 0:
    #     l_edge += step
    #     continue
    
    
    # fig, axs = plt.subplots(nrows=2, sharex=True, gridspec_kw={'hspace': 0.05})

    # fig.set_figwidth(16)
    # fig.set_figheight(8)
    
    # axs[0].set_title(f"#{F_ID}")

    # axs[0].plot(range(l_edge, r_edge), df.d_alpha[l_edge:r_edge], label="D-alpha")
    # axs[0].plot(range(l_edge, r_edge), df.mark_dELM[l_edge:r_edge]*0.6, label="Mark dELM", color="green", alpha=0.7)
    # axs[0].plot(range(l_edge, r_edge), df.mark_LCO[l_edge:r_edge]*0.5, label="Mark LCO", color="blue", alpha=0.7)
    # axs[0].plot(range(l_edge, r_edge), df.mark_EHO[l_edge:r_edge]*0.4, label="Mark EHO", color="orange", alpha=0.7)
    # axs[0].grid(which='major', color='#DDDDDD', linewidth=0.9)
    # axs[0].grid(which='minor', color='#DDDDDD', linestyle=':', linewidth=0.7)
    # axs[0].minorticks_on()
    # axs[0].xaxis.set_minor_locator(AutoMinorLocator(10))
    # axs[0].legend()
    
    # axs[1].plot(range(l_edge, r_edge), mgd_v[l_edge:r_edge] ** 2, label="MGD vert abs", alpha=0.7)
    # axs[1].plot(range(l_edge, r_edge), mgd_r[l_edge:r_edge] ** 2, label="MGD rad abs", alpha=0.7)
    # axs[1].grid(which='major', color='#DDDDDD', linewidth=0.9)
    # axs[1].grid(which='minor', color='#DDDDDD', linestyle=':', linewidth=0.7)
    # axs[1].minorticks_on()
    # axs[1].xaxis.set_minor_locator(AutoMinorLocator(10))
    # axs[1].legend()
    
    # plt.show()

    # plt.close()
    # plt.clf()